In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from pytz import timezone
from datetime import datetime, timedelta
import json
import itertools
from tqdm import tqdm
import os
os.chdir("/content/drive/MyDrive/projects/godaddy-microbusiness-density-forecasting/")

In [2]:
df_train = pd.read_csv("data/train.csv")
df_re = pd.read_csv("data/revealed_test.csv")
df_sub = pd.read_csv("data/sample_submission.csv")
df_train = pd.concat([df_train, df_re]).sort_values(["cfips", "row_id"])

df_train["dcount"] = df_train.groupby(['cfips'])['row_id'].cumcount()
df_train['lastactive'] = df_train.groupby('cfips')['active'].transform('last')

In [3]:
def smape(y_true, y_pred):
  smap = np.zeros(len(y_true))
  num = np.abs(y_true - y_pred)
  dem = ((np.abs(y_true) + np.abs(y_pred)) / 2)
  pos_ind = (y_true!=0)|(y_pred!=0)
  smap[pos_ind] = num[pos_ind] / dem[pos_ind]
  return 100 * np.mean(smap)

In [4]:
ACT_THR = 150
lags = [1, 3, 4, 5] # Nヶ月先予測、3~5はprivate
mults = list(np.round(np.arange(0.9960, 1.0200, 0.0005), 4)) # 補正値の候補
result = []
TS = list(range(35, 41)) # 2022年7月~2022年12月を検証用のデータとした。

for lag in lags:
  # グリッドサーチ
  for mult in mults:
    TS_pred = [t-lag for t in TS] # 2022年7月~2022年12月のlagヶ月前を求める
    y = df_train.query("dcount==@TS & lastactive>@ACT_THR")["active"].to_numpy() # 2022年7月~2022年12月の目的変数を取り出す
    y_pred = df_train.query("dcount==@TS_pred & lastactive>@ACT_THR")["active"].to_numpy() # 2022年7-lag月~2022年12-lag月の目的変数を取り出す
    y_pred = y_pred * mult # 候補補正値multをかけて予測を作る
    a = pd.DataFrame({"y":y, "y_pred":y_pred}).dropna()
    score = smape(a["y"], a["y_pred"])
    result.append([lag, mult, score])

result = pd.DataFrame(result, columns=["lag", "mult", "SMAPE"])
display(result.query("lag==1").sort_values("SMAPE").head(1))
display(result.query("lag==3").sort_values("SMAPE").head(1))
display(result.query("lag==4").sort_values("SMAPE").head(1))
display(result.query("lag==5").sort_values("SMAPE").head(1))

,lag,mult,SMAPE
13,1,1.0025,1.183375


,lag,mult,SMAPE
78,3,1.0105,2.109576


,lag,mult,SMAPE
133,4,1.0135,2.511625


,lag,mult,SMAPE
189,5,1.017,2.897837


In [5]:
# 12月 -> N月で調整したもの
params = {1:1.0025, 3:1.0105, 4:1.0135, 5:1.017}

now = datetime.now().astimezone(timezone('Asia/Tokyo'))
now = f"{now.year}-{now.month}-{now.day}_{now.hour}:{now.minute}:{now.second}"

df_re = pd.read_csv("data/revealed_test.csv")
df_re['lastactive'] = df_re.groupby('cfips')['active'].transform('last')
df_re["dcount"] = df_re.groupby(['cfips'])['row_id'].cumcount()
last_active = df_re.groupby('cfips', as_index=False)['active'].last()

column_names = ['GEO_ID','NAME','S0101_C01_026E']
df2021 = pd.read_csv('data/2021population.csv', usecols=column_names)
#df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]))
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

test = pd.read_csv('data/test.csv')
test = test.merge(last_active, on='cfips')
display(test.head())
# update population
test['microbusiness_density'] = test['active'] / test['cfips'].map(adult2021) * 100
test["month"] = test["row_id"].apply(lambda x:x.split('-')[1]).astype(int)
display(test.head())

for month in [1,3,4,5]:
  mult = params[month]
  mask = (test["month"]==month) & (test['active'] >= ACT_THR)
  test.loc[mask, "microbusiness_density"] *= params[month]

display(test.head(10))
sub = pd.read_csv('data/sample_submission.csv')
sub = pd.merge(sub, test, on="row_id", how="left").rename({"microbusiness_density_y":"microbusiness_density"}, axis=1)
display(sub.head(10))
sub = sub[["row_id", "microbusiness_density"]]
#sub.to_csv(f"submissions/{now}_mult_by_month.csv", index=None)

,row_id,cfips,first_day_of_month,active
0,1001_2022-11-01,1001,2022-11-01,1475
1,1001_2022-12-01,1001,2022-12-01,1475
2,1001_2023-01-01,1001,2023-01-01,1475
3,1001_2023-02-01,1001,2023-02-01,1475
4,1001_2023-03-01,1001,2023-03-01,1475


,row_id,cfips,first_day_of_month,active,microbusiness_density,month
0,1001_2022-11-01,1001,2022-11-01,1475,3.319231,11
1,1001_2022-12-01,1001,2022-12-01,1475,3.319231,12
2,1001_2023-01-01,1001,2023-01-01,1475,3.319231,1
3,1001_2023-02-01,1001,2023-02-01,1475,3.319231,2
4,1001_2023-03-01,1001,2023-03-01,1475,3.319231,3


,row_id,cfips,first_day_of_month,active,microbusiness_density,month
0,1001_2022-11-01,1001,2022-11-01,1475,3.319231,11
1,1001_2022-12-01,1001,2022-12-01,1475,3.319231,12
2,1001_2023-01-01,1001,2023-01-01,1475,3.327529,1
3,1001_2023-02-01,1001,2023-02-01,1475,3.319231,2
4,1001_2023-03-01,1001,2023-03-01,1475,3.354083,3
5,1001_2023-04-01,1001,2023-04-01,1475,3.364041,4
6,1001_2023-05-01,1001,2023-05-01,1475,3.375658,5
7,1001_2023-06-01,1001,2023-06-01,1475,3.319231,6
8,1003_2022-11-01,1003,2022-11-01,14133,7.935207,11
9,1003_2022-12-01,1003,2022-12-01,14133,7.935207,12


,row_id,microbusiness_density_x,cfips,first_day_of_month,active,microbusiness_density,month
0,1001_2022-11-01,3.817671,1001,2022-11-01,1475,3.319231,11
1,1003_2022-11-01,3.817671,1003,2022-11-01,14133,7.935207,11
2,1005_2022-11-01,3.817671,1005,2022-11-01,248,1.240310,11
3,1007_2022-11-01,3.817671,1007,2022-11-01,229,1.286517,11
4,1009_2022-11-01,3.817671,1009,2022-11-01,822,1.818544,11
5,1011_2022-11-01,3.817671,1011,2022-11-01,81,1.002103,11
6,1013_2022-11-01,3.817671,1013,2022-11-01,337,2.274568,11
7,1015_2022-11-01,3.817671,1015,2022-11-01,2500,2.746830,11
8,1017_2022-11-01,3.817671,1017,2022-11-01,390,1.414684,11
9,1019_2022-11-01,3.817671,1019,2022-11-01,241,1.197337,11
